### Import required packages

In [1]:
from osgeo import gdal, osr, ogr
import matplotlib.pyplot as plt
import numpy as np
import os
import osmnx as ox
import rasterio
from rasterstats import zonal_stats
from rasterio.plot import show
from rasterio.enums import Resampling
import geopandas as gpd
from shapely.geometry import Polygon
import json
import math

### Set constants

In [2]:
# CONSTANTS
BASE_CRS = 4326

### Gridify region and export stats from all raster layers

In [ ]:
regionStats(BASE_CRS=BASE_CRS, LOCS=gridify("Antananarivo, Madagascar", square_size=0.1), DIR='layers/', STATS=['mean'])

/opt/anaconda3/envs/mn_analysis/lib/python3.7/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
/opt/anaconda3/envs/mn_analysis/lib/python3.7/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


[ 47.5255809 -18.9100122  47.5255809 -18.9100122]
Analyzing location: [47.4255809, -18.9100122, 47.5255809, -18.8100122]

Opening layers//basic.tif
Number of Layers: 28
EPSG: 4326


/opt/anaconda3/envs/mn_analysis/lib/python3.7/site-packages/rasterstats/io.py:302: UserWarning: Setting nodata to -999; specify nodata explicitly
  warnings.warn("Setting nodata to -999; specify nodata explicitly")


Complete
Analyzing location: [47.4255809, -19.010012200000002, 47.5255809, -18.9100122]

Opening layers//basic.tif
Number of Layers: 28
EPSG: 4326
Complete
Analyzing location: [47.4255809, -19.110012200000003, 47.5255809, -19.010012200000002]

Opening layers//basic.tif
Number of Layers: 28
EPSG: 4326
Complete
Analyzing location: [47.5255809, -18.9100122, 47.6255809, -18.8100122]

Opening layers//basic.tif
Number of Layers: 28
EPSG: 4326
Complete
Analyzing location: [47.5255809, -19.010012200000002, 47.6255809, -18.9100122]

Opening layers//basic.tif
Number of Layers: 28
EPSG: 4326
Complete
Analyzing location: [47.5255809, -19.110012200000003, 47.6255809, -19.010012200000002]

Opening layers//basic.tif
Number of Layers: 28
EPSG: 4326
Complete
Analyzing location: [47.6255809, -18.9100122, 47.725580900000004, -18.8100122]

Opening layers//basic.tif
Number of Layers: 28
EPSG: 4326


### Gridify region 

In [3]:
def gridify(loc, square_size):
    
     # Use location name as Polygon
    region = ox.geocode_to_gdf(loc)
    crs = {'init': 'epsg:'+str(BASE_CRS)}
    region = region.to_crs(crs=crs)

    # Set region bounds and grid square size
#     print(region.total_bounds)
    xmin,ymin,xmax,ymax = region.total_bounds
    side= square_size

    # Create grid columns and rows
    cols = list(np.arange(xmin-side, xmax+side, side))
    rows = list(np.arange(ymin-side, ymax+side, side))
    rows.reverse()

    # Convert grid columns and rows to polygons
    polygons = []
    for x in cols:
        for y in rows:
            polygons.append( Polygon([(x,y), (x+side, y), (x+side, y-side), (x, y-side)]) )

    # Convert to GDF
    grid = gpd.GeoDataFrame({'geometry':polygons})
    
    # Visualize region 
    ax = region.plot(facecolor='lightblue', edgecolor='lightblue', linewidth=2)
#     grid.plot(ax=ax, facecolor='None', edgecolor='black', linewidth=0.5)
    
    # Convert to loc list
    grid_locs = []
    for index, square in grid.iterrows():
        grid_locs.append(list(square["geometry"].bounds))
#     print(grid_locs)
    return grid_locs

### Retrieve and export raster values for region grid

In [4]:
# Base Coordinate System EPSG code 
# Location center [minx, miny, maxx, maxy]
# Raster directory
# Which sampling statistic/s to output

def regionStats(BASE_CRS, LOCS, DIR, STATS):
    output = {}
    tifs = []
    shps = []

    # Iterate through directory
    for subdir, dirs, files in os.walk(DIR):
        for f in files:
            path = subdir + os.sep + f
            if path.endswith(".tif"):
                tifs.append(path)
            elif path.endswith(".shp"):
                shps.append(path)

    output["Locations"] = LOCS

    for ind, LOC in enumerate(LOCS):

        print("Analyzing location: " + str(LOC))

        # Initialize empty location holder
        loc_output = {}

        # Add constants to location output file
        loc_output["Location"] = LOC
        loc_output["Stats"] = STATS
        loc_output["Bands"] = {}

        # Iterate through tif files
        for ds_path in tifs:
            # Select raster image 
            print("\nOpening " + ds_path)
            ds = gdal.Open(ds_path)

            # Get the number of layers
            print("Number of Layers: " + str(ds.RasterCount))
    #         print(dir(ds.GetRasterBand(0).GetStatistics(True, True))

            # Get source coordinate reference type 
            projection = int(osr.SpatialReference(wkt=ds.GetProjection()).GetAttrValue('AUTHORITY',1))
            print("EPSG: " + str(projection))
            source = osr.SpatialReference()
            source.ImportFromEPSG(projection)

            # Set target coordinate reference 
            target = osr.SpatialReference()
            target.ImportFromEPSG(BASE_CRS)

            # Create point transformation
            transform = osr.CoordinateTransformation(source, target)

            # Get raster coordinates (rc) in source reference 
            rc = getRasterCoords(ds)
    #         print("Raster coordinates in source Ref: " + str(rc))

            # Convert raster coordinates to target reference
            rc_conv = np.array([])
            y,x,z = transform.TransformPoint(rc[0], rc[1])
            rc_conv = np.concatenate((rc_conv, [x, y]))
            y,x,z = transform.TransformPoint(rc[2], rc[3])
            rc_conv = np.concatenate((rc_conv, [x, y]))
    #         print("Raster coordinates in target reference: " + str(rc_conv))

            # Check if raster contains the desired location
            if rasterContainsLoc(LOC, rc_conv):
    
                # Iterate through bands
                for band in range(1, 2):#ds.RasterCount+1):
                    model = rasterio.open(ds_path)
                    crs = {'init': 'epsg:'+str(BASE_CRS)}
                    polygon = gpd.GeoDataFrame(crs=crs, geometry=[returnPolygon(LOC)])
                    place = polygon.to_crs(crs=model.crs)

                    # Optional - Use location name as Polygon
    #                 place = ox.geocode_to_gdf("California")
    #                 place = ox.geocode_to_gdf(polygon)
    #                 place = place.to_crs(crs=model.crs)

                    # Optional - Visualize band 
    #                 ax = place.plot(facecolor='None', edgecolor='red', linewidth=2)
    #                 show((model, band), ax=ax)

                    # Convert raster to array
                    array = model.read(band)
                    affine = model.transform

                    # Check if resample is needed
                    if returnPolygon(LOC).area < getPixelSize(ds, transform)[0]*getPixelSize(ds, transform)[1]:
    #                     print("Resampling image from area " + str(getPixelSize(ds)[0]))
                        upscale_factor = getPixelSize(ds, transform)[0]*getPixelSize(ds, transform)[1] / returnPolygon(LOC).area
#                         print(upscale_factor)
                        array = model.read(
                            band,
                            out_shape=(
                                int(model.height * upscale_factor),
                                int(model.width * upscale_factor)
                            ),
                            resampling=Resampling.bilinear)
                        affine = model.transform * model.transform.scale(
                            (model.width / array.shape[-1]),
                            (model.height / array.shape[-2])
                        )

                    # Computer zonal stats
                    zs = zonal_stats(place, array, affine=affine, stats=STATS) 

                    # Write results to output dict
                    loc_output["Bands"][os.path.basename(ds_path) + "-" + str(band)] = zs[0]

                # Finished processing file bands
                print("Complete")

            # If raster does not contain desired location
            else:
                print("ERROR: Raster does not contain location: " + str(LOC) + ". Will not be included in results output.")

        output[ind] = loc_output

    print("\nWriting all location results to results.json...")
    with open('result.json', 'w') as fp:
        json.dump(output, fp, indent=2)
    print("Complete")

### Get raster resolution

In [5]:
def getPixelSize(ds, transform):
    
    gt = ds.GetGeoTransform()
    return(transform.TransformPoint(gt[1], -gt[5]))

### Get raster corner coordinates

In [6]:
def getRasterCoords(ds):
    
    gt = ds.GetGeoTransform()
    w = ds.RasterXSize
    h = ds.RasterYSize
    
    minx = gt[0]
    maxy = gt[3] 
    miny = gt[3] + w*gt[4] + h*gt[5] 
    maxx = gt[0] + w*gt[1] + h*gt[2]
    
    return[minx, miny, maxx, maxy]

### Check if raster contains desired location square

In [7]:
def rasterContainsLoc(LOC, rc):
    if LOC[0] >= rc[0] and LOC[3] <= rc[3] and LOC[1] >= rc[1] and LOC[3] <= rc[3]:
        return True
    else:
        return False

### Return polygon from coordinate square

In [8]:
def returnPolygon(LOC):
    poly = Polygon([(LOC[0], LOC[1]), (LOC[2], LOC[1]), (LOC[2], LOC[3]), (LOC[0], LOC[3]), (LOC[0], LOC[1])])
    return(poly)